In [1]:
%reset -f
import numpy as np
from scipy.special import eval_genlaguerre, jv
from scipy.optimize import newton
import imageio as iio
import os
import math

# Pixel size
lamda = 0.66e-6  # 修正单位为微米
w0 = 900e-6
pix_size = 9.2e-6

# Parameters
mx = 1920
my = 1152

# Shift parameters
shift_x = 0.2 * mx * pix_size
shift_y = 0 * my * pix_size

# Create coordinate grids with shifts
x = np.linspace(-mx * pix_size / 2 + shift_x, mx * pix_size / 2 + shift_x, mx)
y = np.linspace(-my * pix_size / 2 + shift_y, my * pix_size / 2 + shift_y, my)
X, Y = np.meshgrid(x, y)
r = np.sqrt(X**2 + Y**2)
theta = np.arctan2(Y, X)
z = 4e-6
zr = np.pi * w0**2 / lamda
wz = w0 * np.sqrt(1 + (z / zr)**2)
k0 = 2 * np.pi / lamda

# Function to generate phase pattern
def generate_GS(mx=1920, my=1152, a_scale=5, d=4e-6, s=4e-6,z=4e-6):
    if z == 0:
        Rz = np.inf
    else:
        Rz = (z**2 + zr**2) / z
    l = 0
    p = 0
    L = abs(l)
    phase = np.exp(-1j * l * theta) * np.exp(1j * (k0 * r**2 / (2 * Rz) - (2*p + L + 1) * np.arctan(z / zr)))
    am = (2 * math.factorial(p) / (np.pi * math.factorial(p + L)))**0.5 * \
         (1 / wz) * (np.sqrt(2) * r / wz)**L * np.exp(-(r**2 / wz**2)) * \
         eval_genlaguerre(p, L, 2 * (r**2 / wz**2))
    return am * phase

u = generate_GS()
AM = np.abs(u) / np.max(np.abs(u))
phi = np.angle(u)

# Add grating phase
grating_period= pix_size * 5
gra = (2 *  np.pi * X / grating_period) % (2 * np.pi)
phi = phi + gra - np.pi
# Method F
C = 0.5818

def faeq(f_a_ref, a_ref):
    global C
    return jv(1, f_a_ref) - C * a_ref

def find_f_a_ref(a_ref):
    return newton(faeq, np.ones_like(a_ref) * 1, args=(a_ref,))

fa = find_f_a_ref(AM)
h = fa * np.sin(phi)
h = np.mod(h + np.min(h), 2 * np.pi)

# Create a full black image and place the phase pattern in the center
gh = (h * 255 / (2 * np.pi)).astype(np.uint8)

# Output directory
output_dir = r'C:\Users\11455\Desktop\wkx1\程序\python\相图'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Generate LG mode phase pattern
gs_phase = gh
gs_image_path = os.path.join(output_dir, 'gs_phase zx_shift.bmp')
iio.imwrite(gs_image_path, gs_phase)

print(f"Images saved to {output_dir}")



Images saved to C:\Users\11455\Desktop\wkx1\程序\python\相图
